1. Считайте файл 3.csv и создайте Spark DataFrame.

In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions

# Создаем SparkSession
spark = SparkSession.builder \
    .appName("Read CSV File") \
    .getOrCreate()

# Считываем JSON-файл и создаем DataFrame
df = spark.read.csv("3/3.csv")
df.show()

+----+----+----+----+----+----+----+----+----+----+-----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
| _c0| _c1| _c2| _c3| _c4| _c5| _c6| _c7| _c8| _c9| _c10|_c11|_c12|_c13|_c14|_c15|_c16|_c17|_c18|_c19|_c20|_c21|_c22|_c23|_c24|_c25|_c26|_c27|_c28|_c29|_c30|_c31|_c32|_c33|_c34|_c35|_c36|_c37|_c38|_c39|_c40|_c41|_c42|_c43|_c44|_c45|_c46|_c47|_c48|_c49|_c50|_c51|_c52|_c53|_c54|_c55|
+----+----+----+----+----+----+----+----+----+----+-----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+
| 364| 883|-449| 100|null|-329|-604| -31|-614| 943|  176| -64|-629| 406|-736|-543|  -7|  82| -59|-750|-983|-875| 457|null|-394|-259| 429| 816|  69|-563

2. Подсчитайте количество пропусков

In [2]:

column_names = df.columns

sum_nulls = 0

for col in column_names:
    count = df.filter(df[col] == "null").count()
    sum_nulls += count

sum_nulls


1466

3. Обработайте пропуски: заполните средним по колонке

In [ ]:
import pyspark.sql.functions as sf
import pyspark.pandas as ps
from pyspark.sql.functions import when

column_names = df.columns

for col in column_names:
    avg = df.select(sf.try_avg(col))
    ps_avg = ps.DataFrame(avg)
    value = ps_avg.iloc[0,0]
    df= df.withColumn(col, when(df[col] == 'null', value).otherwise(df[col]))

df.show()


d:\URFU\III_HIOBD\Lab2\130\.venv\lib\site-packages\pyspark\pandas\__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


+----+----+-----+----+-----------------+----+----+-------------------+------------------+----+-----+----+------------------+----+----+----+----+----+----+------------------+----+----+----+------------------+----+----+------------------+----+----+----+----+----+----+------------------+----+----+----+----+----+----+----+----+----+----+----+----+----+-----------------+----+----+----+----+------------------+----+----+----+
| _c0| _c1|  _c2| _c3|              _c4| _c5| _c6|                _c7|               _c8| _c9| _c10|_c11|              _c12|_c13|_c14|_c15|_c16|_c17|_c18|              _c19|_c20|_c21|_c22|              _c23|_c24|_c25|              _c26|_c27|_c28|_c29|_c30|_c31|_c32|              _c33|_c34|_c35|_c36|_c37|_c38|_c39|_c40|_c41|_c42|_c43|_c44|_c45|_c46|             _c47|_c48|_c49|_c50|_c51|              _c52|_c53|_c54|_c55|
+----+----+-----+----+-----------------+----+----+-------------------+------------------+----+-----+----+------------------+----+----+----+----+----+----+

4. Произведите фильтрацию данных: оставьте строки, сумма которых < 0

In [4]:

df_filtered = df.withColumn("row_sum", sum(df[col] for col in df.columns)).filter("row_sum < 0")
df_filtered.show()

+-----+----+-----+----+-----------------+----+----+-------------------+------------------+----+-----+----+------------------+----+----+----+----+----+----+------------------+----+----+----+------------------+----+----+------------------+----+----+----+----+----+----+------------------+----+----+----+----+----+-----+----+----+----+----+----+----+----+-----------------+----+----+----+----+------------------+----+----+----+-------------------+
|  _c0| _c1|  _c2| _c3|              _c4| _c5| _c6|                _c7|               _c8| _c9| _c10|_c11|              _c12|_c13|_c14|_c15|_c16|_c17|_c18|              _c19|_c20|_c21|_c22|              _c23|_c24|_c25|              _c26|_c27|_c28|_c29|_c30|_c31|_c32|              _c33|_c34|_c35|_c36|_c37|_c38| _c39|_c40|_c41|_c42|_c43|_c44|_c45|_c46|             _c47|_c48|_c49|_c50|_c51|              _c52|_c53|_c54|_c55|            row_sum|
+-----+----+-----+----+-----------------+----+----+-------------------+------------------+----+-----+----+----

5. Подсчитайте статистику по каждой колонке и сохранить в отдельный df.

In [5]:
df_filtered = df_filtered.drop("row_sum")

stat = df_filtered.describe()

stat.show()

+-------+------------------+------------------+------------------+------------------+-------------------+-------------------+-------------------+-------------------+------------------+------------------+-------------------+------------------+-------------------+-------------------+-------------------+------------------+------------------+-------------------+------------------+-------------------+-------------------+------------------+-------------------+-------------------+-------------------+------------------+-------------------+-------------------+-------------------+-----------------+------------------+------------------+-----------------+------------------+-------------------+------------------+-------------------+-------------------+-------------------+------------------+-------------------+------------------+------------------+-----------------+-----------------+------------------+------------------+------------------+------------------+------------------+------------------+----

6. Запишите полученный df со статистикой: r_3.csv

In [6]:
output_path = "3/r_3.csv"

stat.toPandas().to_csv(output_path)